In [1]:
import scipy.sparse as sparse
import numpy as np
import control
from mpc_control.controller import MPCController

In [9]:
''' Pendulum Dynamics in States Space Equation '''
    
M = 20  # Cart mass
m = 2  # Pendulum mass
b = 0.1  # Coefficient of friction for cart
l = 0.5  # Length to pendulum center of mass
I = (m*l**2)*(1/3)  # Mass moment of inertia of the pendulum
g = 9.8  # Gravity
dt = 0.2 # Time step


p = I*(M+m)+M*m*l**2

A = np.array([[0,      1,              0,            0],
    [0, -(I+m*l**2)*b/p,  (m**2*g*l**2)/p, 0],
    [0,      0,              0,            1],
    [0, -(m*l*b)/p,       m*g*l*(M+m)/p,   0]])

B = np.array([[0],
    [(I+m*l**2)/p],
    [0],
    [m*l/p]])

C = np.array([[1, 0, 0, 0],
        [0, 0, 1, 0]])

D = np.array([[0],
        [0]])

sys = control.StateSpace(A, B, C, D)
sys_discrete = control.c2d(sys, dt, method='zoh')

A_zoh = np.array(sys_discrete.A)
B_zoh = np.array(sys_discrete.B)

''' Model Predictive Control implementation using State Space Equation '''

nx, nu = B_zoh.shape
Q = sparse.diags([10.0, 10.0, 10.0, 10.0]).toarray()
R = np.array([[0.1]])

xr = np.array([2.0, 0.0, 0.0, 0.0]).astype(float)  # Desired states
# xr *= -1.0
N = 20 # length of horizon
dt = 0.01 # time step


In [17]:
controller = MPCController(A_zoh, B_zoh, Np=N, Qx=Q, Qu=R, xref=xr)
controller.setup()

<Block Sparse Row sparse matrix of dtype 'float64'
	with 80 stored elements (blocksize=4x1) and shape (84, 20)>
  Coords	Values
  (4, 0)	0.0009788635044315891
  (5, 0)	0.009823504254280628
  (6, 0)	0.001541513775626132
  (7, 0)	0.016214979944616913
  (8, 1)	0.0009788635044315891
  (9, 1)	0.009823504254280628
  (10, 1)	0.001541513775626132
  (11, 1)	0.016214979944616913
  (12, 2)	0.0009788635044315891
  (13, 2)	0.009823504254280628
  (14, 2)	0.001541513775626132
  (15, 2)	0.016214979944616913
  (16, 3)	0.0009788635044315891
  (17, 3)	0.009823504254280628
  (18, 3)	0.001541513775626132
  (19, 3)	0.016214979944616913
  (20, 4)	0.0009788635044315891
  (21, 4)	0.009823504254280628
  (22, 4)	0.001541513775626132
  (23, 4)	0.016214979944616913
  (24, 5)	0.0009788635044315891
  (25, 5)	0.009823504254280628
  (26, 5)	0.001541513775626132
  (27, 5)	0.016214979944616913
  (28, 6)	0.0009788635044315891
  :	:
  (59, 13)	0.016214979944616913
  (60, 14)	0.0009788635044315891
  (61, 14)	0.009823504254

In [61]:

JX_ON = True
JU_ON = True
JDU_ON = True
SOFT_ON = True



## compute the matrices required for Quadratic Program

In [ ]:
def compute_QP_matrices():
    Np = 5
    Nc = 5 
    nx = 4
    nu = 1
    eps_feas=1e6
    Qx = sparse.diags([10., 5., 100., 5.])
    QxN = Qx.copy()
    Qu = sparse.diags([0.1]) 
    QDu = np.zeros((nu,nu))
    xref = np.array([0., 0., 0., 0.])
    uref = np.zeros(nu)
    uminus1 = uref
    Qeps = eps_feas * sparse.eye(nx)
    # Ad = Ad
    # Bd = Bd
    # x0 = x0
    # xmin = xmin
    # xmax = xmax
    # umin = umin
    # umax = umax
    # Dumin = Dumin
    # Dumax = Dumax
    # Qeps = Qeps

    # casting the MPC problem to a Quadratic Program 
    P_X = sparse.csc_matrix(((Np+1)*nx, (Np+1)*nx))
    q_X = np.zeros((Np+1)*nx)  # x_N
    J_CNST = 0.0
    # for penalizing the states
    if JX_ON:
        P_X += sparse.block_diag([sparse.kron(sparse.eye(Np), Qx),QxN])   # quadratic part 
        q_X += np.hstack([np.kron(np.ones(Np), -Qx.dot(xref)),-QxN.dot(xref)])  # linear part
    else:
        pass
    # for penalizing the control inputs
    P_U = sparse.csc_matrix((Nc*nu, Nc*nu))
    q_U = np.zeros(Nc*nu)
    if JU_ON:
        J_CNST += 1/2*Np*(uref.dot(Qu.dot(uref)))
        if Nc == Np:
            P_U += sparse.kron(sparse.eye(Nc), Qu)
            q_U += np.kron(np.ones(Nc), -Qu.dot(uref))
    else:
        pass
    # for penalizing change in control inputs
    if JDU_ON:
            J_CNST += 1/2*uminus1.dot((QDu).dot(uminus1))
            iDu = 2 * np.eye(Nc) - np.eye(Nc, k=1) - np.eye(Nc, k=-1)
            iDu[Nc - 1, Nc - 1] = 1
            P_U += sparse.kron(iDu, QDu)
            q_U += np.hstack([-QDu.dot(uminus1),np.zeros((Nc - 1) * nu)]) 
    else:
        pass 
    # introducing soft constraints 
    if SOFT_ON:
            P_eps = sparse.kron(np.eye((Np+1)), Qeps)
            q_eps = np.zeros((Np+1)*nx)


    # linear dynamics x_k+1 = Ax_k + Bu_k
    Ax = sparse.kron(sparse.eye(Np + 1), -sparse.eye(nx)) + sparse.kron(sparse.eye(Np + 1, k=-1), Ad)







    
    


        





In [63]:
compute_QP_matrices()

In [32]:
np.ones(Np)

array([1., 1., 1., 1., 1.])

In [31]:
Np = 5
Nc = 5 
nx = 4
nu = 1
Qx = sparse.diags([10., 5., 100., 5.])
QxN = Qx.copy()
Qu = sparse.diags([0.1]) 
# QDu = QDu
xref = np.array([0., 0., 0., 0.])
-Qx.dot(xref)

array([-0., -0., -0., -0.])

In [33]:
np.kron(np.ones(Np), -Qx.dot(xref))

array([-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,
       -0., -0., -0., -0., -0., -0., -0.])

### inequality constraints

In [9]:
Nc = 2
Np = 2
umin = 3
nu = 1
nx = 2

In [6]:
sparse.eye(6).toarray()

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1.]])

In [11]:
sparse.csc_matrix(((Np+1)*nx, Nc*nu)).toarray()

array([[0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]])

In [15]:
n_eps = 6

### state constraint matrix

In [21]:
Aineq_x = sparse.hstack([sparse.eye((Np + 1) * nx), sparse.csc_matrix(((Np+1)*nx, Nc*nu))])
Aineq_x.toarray()

array([[1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.]])

In [23]:
Aineq_x = sparse.hstack([Aineq_x, sparse.eye(n_eps)])
Aineq_x.toarray()

array([[1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [27]:
xmin = [2,1]
xmax = [3,2]

In [28]:
lineq_x = np.kron(np.ones(Np + 1), xmin) 
uineq_x = np.kron(np.ones(Np + 1), xmax) 
lineq_x

array([2., 1., 2., 1., 2., 1.])

### input constraint matrix

In [20]:
Aineq_u = sparse.hstack([sparse.csc_matrix((Nc*nu, (Np+1)*nx)), sparse.eye(Nc * nu)])
Aineq_u.toarray()

array([[0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1.]])

In [18]:

Aineq_u = sparse.hstack([Aineq_u, sparse.csc_matrix((Aineq_u.shape[0], n_eps))]) 
Aineq_u.toarray()

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [4]:
lineq_u = np.kron(np.ones(Nc), umin)   
lineq_u

array([3., 3.])

### delta u constraint

In [31]:
sparse.hstack([np.zeros((nu, (Np + 1) * nx)), sparse.eye(nu), np.zeros((nu, (Nc - 1) * nu))]).toarray()

array([[0., 0., 0., 0., 0., 0., 1., 0.]])

In [33]:
sparse.hstack([np.zeros((Nc * nu, (Np+1) * nx)), -sparse.eye(Nc * nu) + sparse.eye(Nc * nu, k=1)]).toarray()

array([[ 0.,  0.,  0.,  0.,  0.,  0., -1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.]])

In [29]:
Aineq_du = sparse.vstack([sparse.hstack([np.zeros((nu, (Np + 1) * nx)), sparse.eye(nu), np.zeros((nu, (Nc - 1) * nu))]),  # for u0 - u-1
                                  sparse.hstack([np.zeros((Nc * nu, (Np+1) * nx)), -sparse.eye(Nc * nu) + sparse.eye(Nc * nu, k=1)])  # for uk - uk-1, k=1...Np
                                  ]
                                 )
Aineq_du.toarray()

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  1.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0., -1.]])

### dynamics constraint

In [61]:
Ad = [[1,2],[2,4]]

In [62]:
Ax = sparse.kron(sparse.eye(Np + 1), -sparse.eye(nx)) + sparse.kron(sparse.eye(Np + 1, k=-1), Ad)
Ax.toarray()

array([[-1.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.],
       [ 1.,  2., -1.,  0.,  0.,  0.],
       [ 2.,  4.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  1.,  2., -1.,  0.],
       [ 0.,  0.,  2.,  4.,  0., -1.]])

In [40]:
sparse.eye(4).toarray()

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

### basics

In [42]:
sparse.kron(sparse.eye(Np + 1), -sparse.eye(nx)).toarray()

array([[-1.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  0.,  0.,  0., -1.]])

In [44]:
sparse.eye(Np + 1, k=-1).toarray()

array([[0., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.]])

In [46]:
iBu = sparse.vstack([
    sparse.csc_matrix((1, Nc)),  # Row of zeros for k=0
    sparse.eye(Nc)               # Identity matrix for k=1 to Nc
])
iBu.toarray()

array([[0., 0.],
       [1., 0.],
       [0., 1.]])

In [55]:
Bd = [[1],[2]]

In [56]:
Bu = sparse.kron(iBu, Bd)
Bu.toarray()

array([[0., 0.],
       [0., 0.],
       [1., 0.],
       [2., 0.],
       [0., 1.],
       [0., 2.]])

In [58]:
Aeq_dyn = sparse.hstack([Ax, Bu])
Aeq_dyn.toarray()

array([[-1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0., -1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  2., -1.,  0.,  0.,  0.,  1.,  0.],
       [ 2.,  4.,  0., -1.,  0.,  0.,  2.,  0.],
       [ 0.,  0.,  1.,  2., -1.,  0.,  0.,  1.],
       [ 0.,  0.,  2.,  4.,  0., -1.,  0.,  2.]])

In [67]:
x0 = 1

In [68]:
leq_dyn = np.hstack([-x0, np.zeros(Np * nx)])
leq_dyn

array([-1.,  0.,  0.,  0.,  0.])